<a href="https://colab.research.google.com/github/PhilippMatthes/diplom/blob/master/src/shl-deep-learning-timeseries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using a deep CNN to directly classify SHL timeseries data

The following notebook contains code to classify SHL timeseries data with deep convolutional neural networks. This is devided into the following steps:

1. Download the SHL dataset.
2. Preprocess the SHL dataset into features and make it readable efficiently by our training engine.
3. Define one or multiple ml models.
4. Train the model(s) and utilize grid search to find the best configuration.
5. Export the models and their training parameters for later analysis.

## Step 1: Download the SHL Dataset

The SHL dataset is very big, so we will need to free up some disk space on colab, first.

In [ ]:
!rm -rf /usr/local/lib/python2.7
!rm -rf /swift
!rm -rf /usr/local/lib/python3.6/dist-packages/torch
!rm -rf /usr/local/lib/python3.6/dist-packages/pystan
!rm -rf /usr/local/lib/python3.6/dist-packages/spacy
!rm -rf /tensorflow-1.15.2/

Next, get our base repo so that we can use predefined architectures and pretrained scalers.

In [ ]:
!git clone https://github.com/philippmatthes/diplom

Cloning into 'diplom'...
remote: Enumerating objects: 1797, done.
remote: Counting objects: 100% (1134/1134), done.
remote: Compressing objects: 100% (769/769), done.
remote: Total 1797 (delta 570), reused 832 (delta 307), pack-reused 663
Receiving objects: 100% (1797/1797), 34.69 MiB | 23.92 MiB/s, done.
Resolving deltas: 100% (946/946), done.


Switch to our src dir for further processing. This command is specific to Google Colab, so it might not work on your local Jupyter Notebook instance.

Additionally, we create the dataset dir in which our dataset will be downloaded next.

In [ ]:
%cd /content/diplom/src
!mkdir shl-dataset

/content/diplom/src


Download the SHL dataset from the shl server. This might take some time, on Google Colab its approx. 45 minutes. You can also mount your Google Drive if you have enough space available.

In [ ]:
!wget -nc -O shl-dataset/challenge-2019-user1_torso.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_torso.zip
!wget -nc -O shl-dataset/challenge-2019-user1_bag.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_bag.zip
!wget -nc -O shl-dataset/challenge-2019-user1_hips.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_hips.zip
!wget -nc -O shl-dataset/challenge-2020-user1_hand.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2020/challenge-2020-train_hand.zip
!wget -nc -O shl-dataset/challenge-2020-users23_torso_bag_hips_hand.zip http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2020/challenge-2020-validation.zip

--2021-08-22 09:18:00--  http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_torso.zip
Resolving www.shl-dataset.org (www.shl-dataset.org)... 37.187.125.22
Connecting to www.shl-dataset.org (www.shl-dataset.org)|37.187.125.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5852446972 (5.5G) [application/zip]
Saving to: ‘shl-dataset/challenge-2019-user1_torso.zip’

shl-dataset/challen 100%[===================>]   5.45G  9.48MB/s    in 9m 58s  

2021-08-22 09:27:59 (9.33 MB/s) - ‘shl-dataset/challenge-2019-user1_torso.zip’ saved [5852446972/5852446972]

--2021-08-22 09:27:59--  http://www.shl-dataset.org/wp-content/uploads/SHLChallenge2019/challenge-2019-train_bag.zip
Resolving www.shl-dataset.org (www.shl-dataset.org)... 37.187.125.22
Connecting to www.shl-dataset.org (www.shl-dataset.org)|37.187.125.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5628524721 (5.2G) [application/zip]
Saving to: ‘shl-data

Next we unzip our dataset into the running instance's filestorage. *Note that this will probably not work for free subscriptions of Google Colab, since the data is approximately 90-100 GB when extracted.*

In [ ]:
# Unzip training datasets
!unzip -n -d shl-dataset/challenge-2019-user1_torso shl-dataset/challenge-2019-user1_torso.zip
!rm shl-dataset/challenge-2019-user1_torso.zip
!unzip -n -d shl-dataset/challenge-2019-user1_bag shl-dataset/challenge-2019-user1_bag.zip
!rm shl-dataset/challenge-2019-user1_bag.zip
!unzip -n -d shl-dataset/challenge-2019-user1_hips shl-dataset/challenge-2019-user1_hips.zip
!rm shl-dataset/challenge-2019-user1_hips.zip
!unzip -n -d shl-dataset/challenge-2020-user1_hand shl-dataset/challenge-2020-user1_hand.zip
!rm shl-dataset/challenge-2020-user1_hand.zip
!unzip -n -d shl-dataset/challenge-2020-users23_torso_bag_hips_hand shl-dataset/challenge-2020-users23_torso_bag_hips_hand.zip
!rm shl-dataset/challenge-2020-users23_torso_bag_hips_hand.zip

Archive:  shl-dataset/challenge-2019-user1_torso.zip
   creating: shl-dataset/challenge-2019-user1_torso/train/Torso/
  inflating: shl-dataset/challenge-2019-user1_torso/train/Torso/Acc_x.txt  
  inflating: shl-dataset/challenge-2019-user1_torso/train/Torso/Acc_y.txt  
  inflating: shl-dataset/challenge-2019-user1_torso/train/Torso/Acc_z.txt  
  inflating: shl-dataset/challenge-2019-user1_torso/train/Torso/Gra_x.txt  
  inflating: shl-dataset/challenge-2019-user1_torso/train/Torso/Gra_y.txt  
  inflating: shl-dataset/challenge-2019-user1_torso/train/Torso/Gra_z.txt  
  inflating: shl-dataset/challenge-2019-user1_torso/train/Torso/Gyr_x.txt  
  inflating: shl-dataset/challenge-2019-user1_torso/train/Torso/Gyr_y.txt  
  inflating: shl-dataset/challenge-2019-user1_torso/train/Torso/Gyr_z.txt  
  inflating: shl-dataset/challenge-2019-user1_torso/train/Torso/Label.txt  
  inflating: shl-dataset/challenge-2019-user1_torso/train/Torso/LAcc_x.txt  
  inflating: shl-dataset/challenge-2019-user1

## Step 2: Preprocess the data

Explanations will from now on be inside the code, so that you can copy it without losing the contextual information.

In [ ]:
# Change into our project src directory and select the TensorFlow version
# Note: use this as an entrypoint when you already downloaded the dataset

%cd /content/diplom/src
%tensorflow_version 2.x

/content/diplom/src


In [ ]:
# Check configuration and hardware resources

import distutils

import tensorflow as tf

if distutils.version.LooseVersion(tf.__version__) < '2.0':
    raise Exception('This notebook is compatible with TensorFlow 2.0 or higher.')

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
# Define all datasets to train our model on

from pathlib import Path

TRAIN_DATASET_DIRS = [
    Path('shl-dataset/challenge-2019-user1_torso/train/Torso'),
    Path('shl-dataset/challenge-2019-user1_bag/train/Bag'),
    Path('shl-dataset/challenge-2019-user1_hips/train/Hips'),
    Path('shl-dataset/challenge-2020-user1_hand/train/Hand'),
    Path('shl-dataset/challenge-2020-users23_torso_bag_hips_hand/validation/Torso'),         
    Path('shl-dataset/challenge-2020-users23_torso_bag_hips_hand/validation/Bag'),   
    Path('shl-dataset/challenge-2020-users23_torso_bag_hips_hand/validation/Hips'),   
    Path('shl-dataset/challenge-2020-users23_torso_bag_hips_hand/validation/Hand'),   
]

In [ ]:
# Define more useful constants about our dataset

LABEL_ORDER = [
    'Null',
    'Still',
    'Walking',
    'Run',
    'Bike',
    'Car',
    'Bus',
    'Train',
    'Subway',
]

SAMPLE_LENGTH = 500

In [ ]:
# Results from data analysis

CLASS_WEIGHTS = {
    0: 0.0, # NULL label
    1: 1.0021671573438011, 
    2: 0.9985739895697523, 
    3: 2.8994439843842423, 
    4: 1.044135815617944, 
    5: 0.7723505499007343, 
    6: 0.8652474758172704, 
    7: 0.7842127155793044, 
    8: 1.0283208861290594
}

In [ ]:
# Define features for our dataset

from collections import OrderedDict

import numpy as np

# Attributes to load from our dataset
X_attributes = [
    'acc_x', 'acc_y', 'acc_z',
    'mag_x', 'mag_y', 'mag_z',
    'gyr_x', 'gyr_y', 'gyr_z',
    # Parts that are not needed:
    # 'gra_x', 'gra_y', 'gra_z',
    # 'lacc_x', 'lacc_y', 'lacc_z',
    # 'ori_x', 'ori_y', 'ori_z', 'ori_w',
]

# Files within the dataset that contain our attributes
X_files = [
    'Acc_x.txt', 'Acc_y.txt', 'Acc_z.txt',
    'Mag_x.txt', 'Mag_y.txt', 'Mag_z.txt',
    'Gyr_x.txt', 'Gyr_y.txt', 'Gyr_z.txt',
    # Parts that are not needed:
    # 'Gra_x.txt', 'Gra_y.txt', 'Gra_z.txt',
    # 'LAcc_x.txt', 'LAcc_y.txt', 'LAcc_z.txt',
    # 'Ori_x.txt', 'Ori_y.txt', 'Ori_z.txt', 'Ori_w.txt',
]

# Features to generate from our loaded attributes
# Note that `a` is going to be a dict of attribute tracks
X_features = OrderedDict({
    'acc_mag': lambda a: np.sqrt(a['acc_x']**2 + a['acc_y']**2 + a['acc_z']**2),
    'mag_mag': lambda a: np.sqrt(a['mag_x']**2 + a['mag_y']**2 + a['mag_z']**2),
    'gyr_mag': lambda a: np.sqrt(a['gyr_x']**2 + a['gyr_y']**2 + a['gyr_z']**2),
})

# Define where to find our labels for supervised learning
y_file = 'Label.txt'
y_attribute = 'labels'

In [ ]:
# Load pretrained power transformers for feature scaling

import joblib

X_feature_scalers = OrderedDict({})
for feature_name, _ in X_features.items():
    scaler_dir = f'models/shl-scalers/{feature_name}.scaler.joblib'
    scaler = joblib.load(scaler_dir)
    scaler.copy = False # Save memory
    X_feature_scalers[feature_name] = scaler
    print(f'Loaded scaler from {scaler_dir}.')

Loaded scaler from models/shl-scalers/acc_mag.scaler.joblib.
Loaded scaler from models/shl-scalers/mag_mag.scaler.joblib.
Loaded scaler from models/shl-scalers/gyr_mag.scaler.joblib.


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PowerTransformer from version 0.24.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [ ]:
# Load the training and validation data into a high performance datatype

import os
import shutil

from typing import Generator, List, Tuple

from tqdm import tqdm

import pandas as pd

def read_chunks(
    n_chunks: int, 
    X_attr_readers: List[pd.io.parsers.TextFileReader], 
    y_attr_reader: pd.io.parsers.TextFileReader
) -> Generator[Tuple[np.ndarray, np.ndarray], None, None]:
    """
    Read chunks of attribute data and yield it to the caller as tuples of X, y.
    
    This function returns a generator which can be iterated.
    """
    for _ in range(n_chunks):
        # Load raw attribute tracks
        X_raw_attrs = OrderedDict({})
        for X_attribute, X_attr_reader in zip(X_attributes, X_attr_readers):
            X_attr_track = next(X_attr_reader)
            X_attr_track = np.nan_to_num(X_attr_track.to_numpy())
            X_raw_attrs[X_attribute] = X_attr_track

        # Calculate features
        X_feature_tracks = None
        for X_feature_name, X_feature_func in X_features.items():
            X_feature_track = X_feature_func(X_raw_attrs)
            X_feature_track = X_feature_scalers[X_feature_name] \
                .transform(X_feature_track)
            if X_feature_tracks is None:
                X_feature_tracks = X_feature_track
            else:
                X_feature_tracks = np.dstack((X_feature_tracks, X_feature_track))

        # Load labels
        y_attr_track = next(y_attr_reader) # dim (None, sample_length)
        y_attr_track = np.nan_to_num(y_attr_track.to_numpy()) # dim (None, sample_length)
        y_attr_track = y_attr_track[:, 0] # dim (None, 1)

        yield X_feature_tracks, y_attr_track

def count_samples(dataset_dir: Path) -> int:
    """Count the total amount of samples in a shl dataset."""
    n_samples = 0
    # Every file in the dataset has the same length, use the labels file
    with open(dataset_dir / y_file) as f:
        for _ in tqdm(f, desc=f'Counting samples in {dataset_dir}'):
            n_samples += 1
    return n_samples

def create_chunked_readers(
    dataset_dir: Path,
    chunksize: int, 
    xdtype=np.float32, # Use np.float16 with caution, can lead to overflows
    ydtype=np.int
) -> Tuple[List[pd.io.parsers.TextFileReader], pd.io.parsers.TextFileReader]:
    """Initialize chunked csv readers and return them to the caller as a tuple."""
    read_csv_kwargs = { 'sep': ' ', 'header': None, 'chunksize': chunksize }

    X_attr_readers = [] # (dim datasets x readers)
    for filename in X_files:
        X_reader = pd.read_csv(dataset_dir / filename, dtype=xdtype, **read_csv_kwargs)
        X_attr_readers.append(X_reader)
    y_attr_reader = pd.read_csv(dataset_dir / y_file, dtype=ydtype, **read_csv_kwargs)

    return X_attr_readers, y_attr_reader

def export_tfrecords(
    dataset_dir: Path,
    n_chunks=16, # Load dataset in parts to not overload memory
):
    """Transform the given shl dataset into a memory efficient TFRecord."""
    target_dir = f'{dataset_dir}.tfrecord'
    if os.path.isfile(target_dir):
        print(f'{target_dir} already exists.')
        return

    print(f'Exporting to {target_dir}.')

    n_samples = count_samples(dataset_dir)
    chunksize = int(np.floor(n_samples / n_chunks))
    X_attr_readers, y_attr_reader = create_chunked_readers(dataset_dir, chunksize)    

    with tf.io.TFRecordWriter(str(target_dir)) as file_writer:
        with tqdm(total=n_samples, desc=f'Reading samples to {target_dir}') as pbar:
            for X_feature_tracks, y_attr_track in read_chunks(
                n_chunks, X_attr_readers, y_attr_reader
            ):
                for X, y in zip(X_feature_tracks, y_attr_track):
                    X_flat = X.flatten() # TFRecords don't support multidimensional arrays
                    record_bytes = tf.train.Example(features=tf.train.Features(feature={
                        'X': tf.train.Feature(float_list=tf.train.FloatList(value=X_flat)),
                        'y': tf.train.Feature(int64_list=tf.train.Int64List(value=[y])) 
                    })).SerializeToString()
                    file_writer.write(record_bytes)
                pbar.update(chunksize)

for dataset_dir in TRAIN_DATASET_DIRS:
    export_tfrecords(dataset_dir)

Exporting to shl-dataset/challenge-2019-user1_torso/train/Torso.tfrecord.


Counting samples in shl-dataset/challenge-2019-user1_torso/train/Torso: 196072it [00:02, 72192.42it/s]
Reading samples to shl-dataset/challenge-2019-user1_torso/train/Torso.tfrecord: 100%|█████████▉| 196064/196072 [04:10<00:00, 783.93it/s]


Exporting to shl-dataset/challenge-2019-user1_bag/train/Bag.tfrecord.


Counting samples in shl-dataset/challenge-2019-user1_bag/train/Bag: 196072it [00:02, 69826.30it/s]
Reading samples to shl-dataset/challenge-2019-user1_bag/train/Bag.tfrecord: 100%|█████████▉| 196064/196072 [04:18<00:00, 758.70it/s]


Exporting to shl-dataset/challenge-2019-user1_hips/train/Hips.tfrecord.


...

Counting samples in shl-dataset/challenge-2020-users23_torso_bag_hips_hand/validation/Bag: 28789it [00:00, 91037.33it/s]
Reading samples to shl-dataset/challenge-2020-users23_torso_bag_hips_hand/validation/Bag.tfrecord: 100%|█████████▉| 28784/28789 [00:42<00:00, 678.20it/s]


Exporting to shl-dataset/challenge-2020-users23_torso_bag_hips_hand/validation/Hips.tfrecord.


Counting samples in shl-dataset/challenge-2020-users23_torso_bag_hips_hand/validation/Hips: 28789it [00:00, 89821.23it/s]
Reading samples to shl-dataset/challenge-2020-users23_torso_bag_hips_hand/validation/Hips.tfrecord: 100%|█████████▉| 28784/28789 [00:42<00:00, 675.89it/s]


Exporting to shl-dataset/challenge-2020-users23_torso_bag_hips_hand/validation/Hand.tfrecord.


Counting samples in shl-dataset/challenge-2020-users23_torso_bag_hips_hand/validation/Hand: 28789it [00:00, 91006.93it/s]
Reading samples to shl-dataset/challenge-2020-users23_torso_bag_hips_hand/validation/Hand.tfrecord: 100%|█████████▉| 28784/28789 [00:42<00:00, 678.09it/s]


In [ ]:
def decode_tfrecord(record_bytes) -> Tuple[tf.Tensor, tf.Tensor]:
    """Decode a TFRecord example to X, y from its serialized representation."""
    example = tf.io.parse_single_example(record_bytes, {
        'X': tf.io.FixedLenFeature([SAMPLE_LENGTH, len(X_features)], tf.float32),
        'y': tf.io.FixedLenFeature([1], tf.int64)
    })
    return example['X'], example['y']

def create_train_validation_datasets(
    dataset_dirs: List[Path], 
    batch_size=64,
    shuffle_size=20_000, # Must be larger than batch_size
    test_size=256 # In batches
) -> Tuple[tf.data.Dataset, tf.data.Dataset]:
    """
    Create interleaved, shuffled and batched train and 
    validation datasets from the dataset dirs.
    
    Note that this function reads previously generated TFRecords under 
    `dataset_dir.tfrecord` -> use `export_tfrecords` for that.
    """
    tfrecord_dirs = [f'{d}.tfrecord' for d in dataset_dirs]
    print(f'Creating train and validation dataset over {tfrecord_dirs}.')

    # Create a strategy to interleave the datasets
    dataset = tf.data.Dataset.from_tensor_slices(tfrecord_dirs) \
        .interleave(
            lambda x: tf.data.TFRecordDataset(x), 
            cycle_length=batch_size, # Number of input elements that are processed concurrently
            block_length=1 # Return only one element at a time, batching is done later
        ) \
        .shuffle(shuffle_size) \
        .map(decode_tfrecord, num_parallel_calls=tf.data.AUTOTUNE) \
        .batch(batch_size)
    count = sum(1 for _ in dataset)
    print(f'Counted {count * batch_size} samples in combined dataset.')
    training_dataset = dataset.skip(test_size)
    count = sum(1 for _ in training_dataset)
    print(f'Counted {count * batch_size} samples in training dataset.')
    validation_dataset = dataset.take(test_size)
    count = sum(1 for _ in validation_dataset)
    print(f'Counted {count * batch_size} samples in validation dataset.')
    return training_dataset, validation_dataset

train_dataset, validation_dataset = create_train_validation_datasets(TRAIN_DATASET_DIRS)

Creating train and validation dataset over ['shl-dataset/challenge-2019-user1_torso/train/Torso.tfrecord', 'shl-dataset/challenge-2019-user1_bag/train/Bag.tfrecord', 'shl-dataset/challenge-2019-user1_hips/train/Hips.tfrecord', 'shl-dataset/challenge-2020-user1_hand/train/Hand.tfrecord', 'shl-dataset/challenge-2020-users23_torso_bag_hips_hand/validation/Torso.tfrecord', 'shl-dataset/challenge-2020-users23_torso_bag_hips_hand/validation/Bag.tfrecord', 'shl-dataset/challenge-2020-users23_torso_bag_hips_hand/validation/Hips.tfrecord', 'shl-dataset/challenge-2020-users23_torso_bag_hips_hand/validation/Hand.tfrecord'].
Counted 899392 samples in combined dataset.
Counted 883008 samples in training dataset.
Counted 16384 samples in validation dataset.


## Steps 3-5: Defining, training and evaluating models

In [ ]:
# We will use the keras tuner contribution package for a hyperparameter gridsearch

import sys

!{sys.executable} -m pip install keras-tuner -q

     |████████████████████████████████| 96 kB 3.2 MB/s 


In [27]:
# Create a logger that will save our progress, even when
# colab decides to kill our training instance later on

import tempfile

from keras_tuner.engine.logger import Logger
from google.colab.files import download

class ZIPProducer(Logger):
    """
    A helper class to be passed with the `logger` argument of `tuner.search`.
    
    On trial completion, this class will automatically 
    create a zip archive of the progress for later analysis.
    """
    def __init__(self, gridsearch_dir: Path):
        self.gridsearch_dir = gridsearch_dir
    
    def register_tuner(self, tuner_state):
        """Informs the logger that a new search is starting."""
        pass

    def register_trial(self, trial_id, trial_state):
        """Informs the logger that a new Trial is starting."""
        with tempfile.TemporaryDirectory() as tempdir:
            # Copy all files (except the checkpoints which become very large)
            # to a temporary directory and zip them, then download
            files_to_ignore = shutil.ignore_patterns('checkpoints*')
            target = f'{tempdir}/gridsearch'
            shutil.copytree(self.gridsearch_dir, target, ignore=files_to_ignore)
            shutil.make_archive('models/gridsearch', 'zip', target)
            download('models/gridsearch.zip')
        print(f'Saved gridsearch progress under gridsearch.zip -> Make sure to download!')

    def report_trial_state(self, trial_id, trial_state):
        """Gives the logger information about trial status."""
        pass            

    def exit(self):
        pass

In [ ]:
from keras_tuner import Hyperband
from keras_tuner.engine import hypermodel as hm_module

class Tuner(Hyperband):
    """
    A custom hyperband tuner, which circumvents an issue 
    in the implementation of keras tuner - Models seem to 
    start from cold every new epoch, which is clearly unwanted. 

    See: https://github.com/keras-team/keras-tuner/issues/372
    And: https://arxiv.org/pdf/1603.06560.pdf
    """
    def _on_train_begin(self, model, hp, *fit_args, **fit_kwargs):
        prev_trial_id = hp.values['tuner/trial_id'] if 'tuner/trial_id' in hp else None
        if prev_trial_id:
            prev_trial = self.oracle.trials[prev_trial_id]
            best_epoch = prev_trial.best_step
            # the code below is from load_model method of Tuner class
            with hm_module.maybe_distribute(self.distribution_strategy):
                model.load_weights(self._get_checkpoint_fname(
                    prev_trial.trial_id, best_epoch
                ))

In [ ]:
# We will use the kapre contribution package to include STFT layers

!{sys.executable} -m pip install kapre -q

In [ ]:
# Define helper functions for resnet model creation

import kapre

from keras_tuner import HyperParameters
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, applications

def twod_resnet_hypermodel(hp: HyperParameters) -> models.Model:
    """Create a 2d stft hypermodel with the given hyperparameters."""
    input_shape = (SAMPLE_LENGTH, len(X_features))

    architectures = OrderedDict({
        'ResNet50': applications.ResNet50, # 98 MB
        'ResNet101': applications.ResNet101, # 172 MB
        'ResNet152': applications.ResNet152, # 232 MB
        'ResNet50V2': applications.ResNet50V2, # 98 MB
        'ResNet101V2': applications.ResNet101V2, # 171 MB
        'ResNet152V2': applications.ResNet152V2, # 232 MB
    })

    # Hyperparameters
    chosen_architecture_name = hp.Choice(
        '2d_model_architecture', 
        values=list(architectures.keys())
    )
    
    chosen_architecture = architectures[chosen_architecture_name]

    model = models.Sequential([
        # Short-time fourier transform
        kapre.STFT(
            n_fft=100,
            hop_length=5,
            pad_end=False,
            input_data_format='channels_last', 
            output_data_format='channels_last',
            input_shape=input_shape,
            name='stft-layer'
        ),
        kapre.Magnitude(),
        kapre.MagnitudeToDecibel(),

        layers.UpSampling2D(2),

        chosen_architecture(
            include_top=True, input_tensor=None, 
            input_shape=(162, 102, 3), weights=None,
            pooling='avg', classes=len(LABEL_ORDER)
        ),
    ])

    return model

def oned_resnet_hypermodel(hp: HyperParameters) -> models.Model:
    """Create a 1d resnet hypermodel with the given hyperparameters."""
    input_shape = (SAMPLE_LENGTH, len(X_features))
    input_layer = layers.Input(input_shape)

    # Hyperparameters
    base_block_height = hp.Int('1d_base_block_height', 32, 128, step=32)
    blocks_until_size_duplication = hp.Int('1d_blocks_until_size_duplication', 2, 4)
    n_blocks = hp.Int('1d_n_blocks', 2, 6)

    def oned_resnet_block(input_layer: layers.Layer, block_height: int) -> layers.Layer:
        """Create a 1d resnet block with the given block height."""
        conv_kwargs = { 
            'filters': block_height, 
            'padding': 'same', 
            'kernel_regularizer': 'l2',
        }

        conv_x = layers.Conv1D(kernel_size=8, **conv_kwargs)(input_layer)
        conv_x = layers.BatchNormalization()(conv_x)
        conv_x = layers.LeakyReLU(alpha=0.2)(conv_x)

        conv_y = layers.Conv1D(kernel_size=5, **conv_kwargs)(conv_x)
        conv_y = layers.BatchNormalization()(conv_y)
        conv_y = layers.LeakyReLU(alpha=0.2)(conv_y)

        conv_z = layers.Conv1D(kernel_size=3, **conv_kwargs)(conv_y)
        conv_z = layers.BatchNormalization()(conv_z)

        shortcut = layers.Conv1D(kernel_size=1, **conv_kwargs)(input_layer)
        shortcut = layers.BatchNormalization()(shortcut)

        output_block = layers.add([shortcut, conv_z])
        output_block = layers.LeakyReLU(alpha=0.2)(output_block)

        return output_block

    endpoint_layer = input_layer # Will be built now
    for i in range(n_blocks):
        n_filters = (int(np.floor(i / blocks_until_size_duplication)) + 1) * base_block_height
        endpoint_layer = make_resnet_block(endpoint_layer, n_filters)
    
    gap_layer = layers.GlobalAveragePooling1D()(endpoint_layer)
    output_layer = layers.Dense(len(LABEL_ORDER), activation='softmax')(gap_layer)

    model = models.Model(inputs=input_layer, outputs=output_layer)

    return model

def resnet_hypermodel(hp: HyperParameters) -> models.Model:
    """Make a resnet hypermodel"""
    model_type = hp.Choice('model_type', ['2d', '1d'])

    hp_kwargs = { 'parent_name': 'model_type', 'parent_values': [model_type] }
    if model_type == '2d':
        with hp.conditional_scope('model_type', ['2d']):
            model = twod_resnet_hypermodel(hp)
    elif model_type == '1d':
        with hp.conditional_scope('model_type', ['1d']):
            model = oned_resnet_hypermodel(hp)
    else:
        raise ValueError('Unknown meta architecture!')

    model.compile(
        loss='sparse_categorical_crossentropy', # No OHE necessary
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        metrics=['acc']
    )

    return model

In [28]:
tuner = Tuner(
    hypermodel=resnet_hypermodel, 
    objective='val_acc', 
    max_epochs=15, 
    overwrite=False,
    directory='models',
    project_name='shl-resnet-gridsearch',
    logger=ZIPProducer('models/shl-resnet-gridsearch'),
)

tuner.search_space_summary()

INFO:tensorflow:Reloading Oracle from existing project models/shl-resnet-gridsearch/oracle.json
INFO:tensorflow:Reloading Tuner from models/shl-resnet-gridsearch/tuner0.json
Search space summary
Default search space size: 2
model_type (Choice)
{'default': '2d', 'conditions': [], 'values': ['2d', '1d'], 'ordered': False}
2d_model_architecture (Choice)
{'default': 'ResNet50', 'conditions': [{'class_name': 'Parent', 'config': {'name': 'model_type', 'values': ['2d']}}], 'values': ['ResNet50', 'ResNet101', 'ResNet152', 'ResNet50V2', 'ResNet101V2', 'ResNet152V2'], 'ordered': False}


In [ ]:
# Define callbacks for our training

from tensorflow.keras import callbacks

decay_lr = callbacks.ReduceLROnPlateau(
    monitor='val_acc',
    factor=0.5, 
    patience=5, # Epochs
    min_lr=0.00001, 
    verbose=1
)

stop_early = callbacks.EarlyStopping(
    monitor='val_acc', 
    patience=10, # Epochs
    verbose=1
)

In [ ]:
# Keras tuner grid search training

tuner.search(
    train_dataset,
    epochs=15,
    callbacks=[decay_lr, stop_early],
    validation_data=validation_dataset,
    verbose=1,
    shuffle=False, # Shuffling doesn't work with our prefetching
    class_weight=CLASS_WEIGHTS,
)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saved gridsearch progress under gridsearch.zip -> Make sure to download!

Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
model_type        |2d                |2d                
2d_model_archit...|ResNet152V2       |ResNet152         
tuner/epochs      |2                 |2                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |2                 |2                 
tuner/round       |0                 |0                 

Epoch 1/2
   9380/Unknown - 3068s 326ms/step - loss: 0.3644 - acc: 0.8621